In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# replace NaN reject_code with 0
claims_df = pd.read_csv('../data/processed/dim_claims_train.csv').fillna(0)
# type cast reject_code as int for nicer printing
claims_df.loc[:, 'reject_code'] = claims_df['reject_code'].astype(int)

date_df = pd.read_csv('../data/processed/dim_date_train.csv')
pa_df = pd.read_csv('../data/processed/dim_pa_train.csv')
bridge_df = pd.read_csv('../data/processed/bridge_train.csv')

## What is the average rate of approved prior authorizations?

In [3]:
print(f"{round(100 * pa_df['pa_approved'].mean(), 1)}% of PAs ({pa_df['pa_approved'].count()} records) are approved.")

73.5% of PAs (444682 records) are approved.


## What is the aggregate rate of approved prior authorizations, segmented by `correct_diagnosis`, `tried_and_failed`, and `contraindication`?
- A correct diagnosis for the drug prescribed **increases** approval rate by 3.9%.
- Trying and failing a generic alternative **increases** approval rate by 11.1%.
- A drug with a contraindication **decreases** approval rate by 24.5%.

In [4]:
for name in ['correct_diagnosis', 'tried_and_failed', 'contraindication']:
    for outcome, view in pa_df.groupby(name):
        print(f"{round(100 * view['pa_approved'].mean(), 1)}% of PAs ({view['pa_approved'].count()} records) are approved if the {name} is {bool(outcome)}.")
    print('')

70.4% of PAs (89040 records) are approved if the correct_diagnosis is False.
74.3% of PAs (355642 records) are approved if the correct_diagnosis is True.

67.9% of PAs (221940 records) are approved if the tried_and_failed is False.
79.0% of PAs (222742 records) are approved if the tried_and_failed is True.

78.4% of PAs (355368 records) are approved if the contraindication is False.
53.9% of PAs (89314 records) are approved if the contraindication is True.



## How does the payer (`bin`) influence the average rate of prior authorization approval?
* Payer `999001` approves 90.7% of PAs, but has the least amount of PAs (51344).

In [5]:
merged = bridge_df.merge(claims_df, on='dim_claim_id').merge(pa_df, on='dim_pa_id')

for payer, view in merged.groupby('bin'):
    print(f"{round(100 * view['pa_approved'].mean(), 1)}% of PAs ({view['pa_approved'].count()}) are approved with payer of {payer}.")

78.7% of PAs (106801) are approved with payer of 417380.
71.1% of PAs (193722) are approved with payer of 417614.
62.9% of PAs (92815) are approved with payer of 417740.
90.7% of PAs (51344) are approved with payer of 999001.


## How does the drug (`drug`) influence the average rate of prior authorization approval?

In [6]:
for drug, view in merged.groupby('drug'):
    print(f"{round(100 * view['pa_approved'].mean(), 1)}% of PAs ({view['pa_approved'].count()}) are approved with drug of {drug}.")

76.3% of PAs (230732) are approved with drug of A.
75.9% of PAs (123482) are approved with drug of B.
63.1% of PAs (90468) are approved with drug of C.


## How does the reject code (`reject_code`) influence the average rate of prior authorization approval?
- 50% of PAs with reject code 70 (drug not covered by plan and not on formulary) approved.
- 94.8% of PAs with reject code 75 (drug is in formulary but does not have preferred status) approved.
- 88.4% of PAs with reject code 76 (drug is covered, but plan limits have been exceeded) approved.

In [7]:
for reject_code, view in merged.groupby('reject_code'):
    print(f"{round(100 * view['pa_approved'].mean(), 1)}% of PAs ({view['pa_approved'].count()}) are approved with reject code of {reject_code}.")

50.0% of PAs (201599) are approved with reject code of 70.
94.8% of PAs (173935) are approved with reject code of 75.
88.4% of PAs (69148) are approved with reject code of 76.


## How does the combination of payer (`bin`) and reject code (`reject_code`) influence the average rate of prior authorization approval?
- Reject code 70 has the lowest rate of approval for each drug.
- Reject code 75 has the highest rate of approval for each drug.
- Payer `999001` always uses reject code 76 and has a higher rate of approval compared to the other payer with reject code 76 for that specific drug.
- For each drug , each payer *except* `999001` has a single code they use. Each payer also only uses each code once.

In [8]:
split = pd.DataFrame(merged.groupby(['drug', 'bin', 'reject_code'])['pa_approved'].mean())
split.loc[:, 'pa_approved_count'] = merged.groupby(['drug', 'bin', 'reject_code'])['pa_approved'].count()
split = split.rename(columns={'pa_approved': 'pa_approved_percent'})
display(split.sort_index(level=[0, 1, 2]))

pa_approved_percent  pa_approved_count
drug bin    reject_code                                        
A    417380 75                      0.990274              70844
     417614 70                      0.582805             125179
     417740 76                      0.902317               8630
     999001 76                      0.963265              26079
B    417380 76                      0.903797               3503
     417614 75                      0.973740              62872
     417740 70                      0.389096              43966
     999001 76                      0.929305              13141
C    417380 70                      0.330683              32454
     417614 76                      0.636396               5671
     417740 75                      0.832940              40219
     999001 76                      0.760887              12124

## How does the combination of contraindication (`contraindication`), generic failure (`tried_and_failed`), and correct diagnosis (`correct_diagnosis`) influence the average rate of prior authorization approval?
- The magnitude of influence of each feature is consistent with the averages above.

In [9]:
split = pd.DataFrame(merged.groupby(['contraindication', 'tried_and_failed', 'correct_diagnosis'])['pa_approved'].mean())
split.loc[:, 'pa_approved_count'] = merged.groupby(['contraindication', 'tried_and_failed', 'correct_diagnosis'])['pa_approved'].count()
split = split.rename(columns={'pa_approved': 'pa_approved_percent'})

display(split.sort_values('pa_approved_percent', ascending=False))

pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.844382   
                                  0                             0.810968   
                 0                1                             0.738509   
                                  0                             0.698910   
1                1                1                             0.611386   
                                  0                             0.567806   
                 0                1                             0.482996   
                                  0                             0.438940   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                             142304  
                                  0                              35687  
                 0                1                             141959  
                                  0                              35418  
1                1                1                              35799  
                                  0                               8952  
                 0                1                              35580  
                                  0                               8983

## For each drug (`drug`) and payer (`bin`), how does the contraindication (`contraindication`), generic failure (`tried_and_failed`), and correct diagnosis (`correct_diagnosis`) influence the average rate of prior authorization approval?
- The relative magnitude of each affect is the same as seen in the previous result. However, the absolute magnitude is different for each payer/drug combination.

In [10]:
for drug in merged['drug'].unique():
    print(f'-- Drug {drug} --')
    for payer in merged['bin'].unique():
        section = merged.loc[(merged['drug'] == drug) & (merged['bin'] == payer)]
        split = pd.DataFrame(section.groupby(['contraindication', 'tried_and_failed', 'correct_diagnosis'])['pa_approved'].mean())
        split.loc[:, 'pa_approved_count'] = section.groupby(['contraindication', 'tried_and_failed', 'correct_diagnosis'])['pa_approved'].count()
        split = split.rename(columns={'pa_approved': 'pa_approved_percent'})
        
        reject_code = section['reject_code'].unique()[0]
        print(f'PA approval rate for payer {payer} (reject_code {reject_code}) and drug {drug}')
        display(split.sort_values('pa_approved_percent', ascending=False))

-- Drug A --
PA approval rate for payer 417380 (reject_code 75) and drug A


pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.999067   
                                  0                             0.998938   
                 0                1                             0.996427   
                                  0                             0.994802   
1                1                1                             0.981930   
                                  0                             0.971034   
                 0                1                             0.946662   
                                  0                             0.927978   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                              22500  
                                  0                               5650  
                 0                1                              22667  
                                  0                               5771  
1                1                1                               5700  
                                  0                               1450  
                 0                1                               5662  
                                  0                               1444

PA approval rate for payer 417740 (reject_code 76) and drug A


pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.972973   
                                  0                             0.962798   
                 0                1                             0.930772   
                                  0                             0.898827   
1                1                1                             0.828854   
                                  0                             0.787565   
                 0                1                             0.656904   
                                  0                             0.530864   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                               2738  
                                  0                                672  
                 0                1                               2759  
                                  0                                682  
1                1                1                                707  
                                  0                                193  
                 0                1                                717  
                                  0                                162

PA approval rate for payer 999001 (reject_code 76) and drug A


...

pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.942748   
                                  0                             0.922303   
                 0                1                             0.852971   
                                  0                             0.818462   
1                1                1                             0.700189   
                                  0                             0.647668   
                 0                1                             0.503315   
                                  0                             0.430446   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                              12838  
                                  0                               3282  
                 0                1                              12977  
                                  0                               3239  
1                1                1                               3182  
                                  0                                772  
                 0                1                               3167  
                                  0                                762

PA approval rate for payer 999001 (reject_code 76) and drug C


pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.900074   
                                  0                             0.881466   
                 0                1                             0.775867   
                                  0                             0.718191   
1                1                1                             0.581560   
                                  0                             0.488889   
                 0                1                             0.380849   
                                  0                             0.340164   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                               4033  
                                  0                                928  
                 0                1                               3837  
                                  0                                951  
1                1                1                                987  
                                  0                                225  
                 0                1                                919  
                                  0                                244

PA approval rate for payer 417614 (reject_code 76) and drug C


pa_approved_percent  \
contraindication tried_and_failed correct_diagnosis                        
0                1                1                             0.838052   
                                  0                             0.792541   
                 0                1                             0.640284   
                                  0                             0.586134   
1                1                1                             0.440449   
                                  0                             0.333333   
                 0                1                             0.185654   
                                  0                             0.106061   

                                                     pa_approved_count  
contraindication tried_and_failed correct_diagnosis                     
0                1                1                               1766  
                                  0                                429  
                 0                1                               1832  
                                  0                                476  
1                1                1                                445  
                                  0                                117  
                 0                1                                474  
                                  0                                132